## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [8]:
import keras
import numpy as np
from keras.datasets import cifar10, cifar100
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.models import Model
from keras import optimizers, regularizers
from keras import backend as K

#from tensorflow.python.keras.layers import Flatten, Dropout
#from tensorflow.python.keras.applications.resnet50 import ResNet50

In [9]:
num_classes = 10

subtract_pixel_mean = True

# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data. # 将数据格式化到0~1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)


# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean
    
# Convert class vectors to binary class matrices.
# 进行one-hot编码
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)


In [25]:
stack_n            = 1
layers             = 6 * stack_n + 2
num_classes        = 10
batch_size         = 128
epochs             = 100
iterations         = 10000 // batch_size + 1
weight_decay       = 1e-4

log_filepath = './my_resnet_32/'

In [12]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 81:
        return 0.1
    if epoch < 122:
        return 0.01
    return 0.001

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, 10) # number of classes
y_test = keras.utils.to_categorical(y_test,10)# number of classes

# color preprocessing
x_train, x_test = color_preprocessing(x_train, x_test)

In [13]:
def residual_block(x,o_filters,increase=False):
    stride = (1,1)
    if increase:
        stride = (2,2)
        
    conv_1 = Conv2D(o_filters,kernel_size=(3,3),strides=stride,padding='same',
                    kernel_initializer="he_normal",
                    kernel_regularizer=regularizers.l2(weight_decay))(x)
    
    o1 = Activation('relu')(BatchNormalization(momentum=0.9, epsilon=1e-5)(conv_1))
    
    conv_2 = Conv2D(o_filters,kernel_size=(3,3),strides=(1,1),padding='same',
                    kernel_initializer="he_normal",
                    kernel_regularizer=regularizers.l2(weight_decay))(o1)
    
    o2  = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv_2)

    if increase:
        projection = Conv2D(o_filters,kernel_size=(1,1),strides=(2,2),padding='same',
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(weight_decay))(x)
        block = add([o2, projection])
        block = Activation('relu')(block)
    else:
        block = add([o2, x])
        block = Activation('relu')(block)
    return block

In [18]:
def residual_network(img_input,classes_num=10,stack_n=5):
    # build model ( total layers = stack_n * 3 * 2 + 2 )
    # stack_n = 5 by default, total layers = 32
    # input: 32x32x3 output: 32x32x16
    x = Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same',
               kernel_initializer="he_normal",
               kernel_regularizer=regularizers.l2(weight_decay))(img_input)

    # input: 32x32x16 output: 32x32x16
    for _ in range(stack_n):
        x = residual_block(x,16,False)

    # input: 32x32x16 output: 16x16x32
    x = residual_block(x,32,True)
    for _ in range(1,stack_n):
        x = residual_block(x,32,False)
    
    # input: 16x16x32 output: 8x8x64
    x = residual_block(x,64,True)
    for _ in range(1,stack_n):
        x = residual_block(x,64,False)

    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    # input: 64 output: 10
    x = Dense(classes_num,activation='softmax',kernel_initializer="he_normal",
              kernel_regularizer=regularizers.l2(weight_decay))(x)
    return x

In [21]:
# build network
img_input = Input(shape=(32,32,3))
output    = residual_network(img_input,10,stack_n)  # 5
model     = Model(img_input, output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 32, 32, 16)   448         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 32, 32, 16)   2320        conv2d_37[0][0]                  
__________________________________________________________________________________________________
batch_normalization_34 (BatchNo (None, 32, 32, 16)   64          conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation

In [26]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# dump checkpoint if you need.(add it to cbks)
# ModelCheckpoint('./checkpoint-{epoch}.h5', save_best_only=False, mode='auto', period=10)

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)

datagen.fit(x_train)

# start training
history = model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                     steps_per_epoch=iterations,
                     epochs=epochs,
                     callbacks=cbks,
                     validation_data=(x_test, y_test))
model.save('my_resnet_32/my_resnet_32.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
79/79 [==============================] - 126s 2s/step - loss: 1.7568 - acc: 0.3685 - val_loss: 1.8591 - val_acc: 0.3727
Epoch 2/100
79/79 [==============================] - 121s 2s/step - loss: 1.5620 - acc: 0.4454 - val_loss: 1.4812 - val_acc: 0.4862
Epoch 3/100
79/79 [==============================] - 120s 2s/step - loss: 1.3735 - acc: 0.5262 - val_loss: 1.3914 - val_acc: 0.5322
Epoch 4/100
79/79 [==============================] - 119s 2s/step - loss: 1.2554 - acc: 0.5726 - val_loss: 1.3157 - val_acc: 0.5498
Epoch 5/100
79/79 [==============================] - 119s 2s/step - loss: 1.2034 - acc: 0.5979 - val_loss: 1.2993 - val_acc: 0.5750
Epoch 6/100
79/79 [==============================] - 120s 2s/step - loss: 1.1405 - acc: 0.6183 - val_loss: 1.2881 - val_acc: 0.5731
Epoch 7/100
79/79 [==============================] - 122s 2s/step - loss: 1.1042 - acc: 0.6337 - val_loss: 1.2555 - val_acc: 0.5831
Epoch 8/100
79/79 [==============================] - 121s 2s/step - loss: 1.

79/79 [==============================] - 121s 2s/step - loss: 0.6943 - acc: 0.8080 - val_loss: 0.8379 - val_acc: 0.7705
Epoch 63/100
79/79 [==============================] - 130s 2s/step - loss: 0.6888 - acc: 0.8124 - val_loss: 0.8525 - val_acc: 0.7647
Epoch 64/100
79/79 [==============================] - 125s 2s/step - loss: 0.6817 - acc: 0.8129 - val_loss: 0.7951 - val_acc: 0.7794
Epoch 65/100
79/79 [==============================] - 125s 2s/step - loss: 0.6630 - acc: 0.8184 - val_loss: 0.7951 - val_acc: 0.7831
Epoch 66/100
79/79 [==============================] - 124s 2s/step - loss: 0.6838 - acc: 0.8137 - val_loss: 0.7543 - val_acc: 0.7956
Epoch 67/100
79/79 [==============================] - 127s 2s/step - loss: 0.6867 - acc: 0.8146 - val_loss: 0.7880 - val_acc: 0.7791
Epoch 68/100
79/79 [==============================] - 124s 2s/step - loss: 0.6785 - acc: 0.8154 - val_loss: 0.8108 - val_acc: 0.7757
Epoch 69/100
79/79 [==============================] - 124s 2s/step - loss: 0.6916 